<a href="https://colab.research.google.com/github/Code-of-Asura-King/Implementation-of-ChatBOT-Week1/blob/main/ImplementationOfChatBOT_Week1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Implementation of ChatBOT Using NLP**

In [1]:
#installing necessary modules
!pip install nltk scikit-learn streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.7 MB/s eta 0:00:00


In [2]:
# Importing necessary libraries
import nltk
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [3]:
ssl._create_default_https_context = ssl._create_unverified_context #overrides Python’s default HTTPS context with one that does not verify SSL certificates.
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
intents = [
    {
        "tag": "greeting",
        "patterns": [
            "Hi", "Hello", "Hey", "Howdy", "What's up",
            "Greetings", "Good morning", "Good afternoon", "Good evening"
        ],
        "responses": [
            "Hello there!", "Hi!", "Greetings!",
            "Hey! How can I help you today?", "Hi there, what can I do for you?"
        ]
    },
    {
        "tag": "goodbye",
        "patterns": [
            "Bye", "Goodbye", "See you later",
            "Talk to you later", "Catch you later", "See ya"
        ],
        "responses": [
            "Goodbye!", "See you soon!",
            "Talk to you later!", "Bye! Have a great day!"
        ]
    },
    {
        "tag": "thanks",
        "patterns": [
            "Thanks", "Thank you", "Thanks a lot",
            "I appreciate it", "Much appreciated"
        ],
        "responses": [
            "You're welcome!", "No problem!",
            "Anytime!", "My pleasure!", "Glad I could help!"
        ]
    },
    {
        "tag": "about",
        "patterns": [
            "What can you do?", "Who are you?",
            "Tell me about yourself", "What is your purpose?"
        ],
        "responses": [
            "I'm a chatbot designed to assist you with various queries and tasks.",
            "I can provide information, answer questions, and help with everyday tasks.",
            "I'm here to help you!"
        ]
    },
    {
        "tag": "help",
        "patterns": [
            "I need help", "Can you help me?", "Help",
            "Assistance required", "I'm lost"
        ],
        "responses": [
            "Sure, I'm here to help. What do you need assistance with?",
            "How can I assist you today?",
            "Please tell me what you need help with."
        ]
    },
    {
        "tag": "age",
        "patterns": [
            "How old are you?", "What's your age?", "When were you created?"
        ],
        "responses": [
            "I don't have an age like humans do. I'm a digital entity.",
            "I'm as old as the internet!",
            "Age is just a number for me."
        ]
    },
    {
        "tag": "weather",
        "patterns": [
            "What's the weather like?", "How's the weather today?",
            "Is it going to rain?", "Tell me the weather"
        ],
        "responses": [
            "I'm sorry, I can't provide real-time weather information.",
            "I don't have current weather data, but you can check a weather website or app for up-to-date info.",
            "I suggest using a weather service for the latest updates."
        ]
    },
    {
        "tag": "budget",
        "patterns": [
            "How can I make a budget?", "What's a good budgeting strategy?",
            "How do I create a budget?", "Budgeting tips"
        ],
        "responses": [
            "To create a budget, start by listing your income and expenses. Track your spending for a month and identify areas to save.",
            "A popular method is the 50/30/20 rule: 50% on essentials, 30% on discretionary spending, and 20% on savings or debt repayment.",
            "Budgeting is all about planning your finances. Consider your goals and adjust your spending accordingly."
        ]
    },
    {
        "tag": "credit_score",
        "patterns": [
            "What is a credit score?", "How do I check my credit score?",
            "How can I improve my credit score?"
        ],
         "responses": ["To make a budget, start by tracking your income and expenses. Then, allocate your income towards essential expenses like rent, food, and bills. Next, allocate some of your income towards savings and debt repayment. Finally, allocate the remainder of your income towards discretionary expenses like entertainment and hobbies.",
                       "A good budgeting strategy is to use the 50/30/20 rule. This means allocating 50% of your income towards essential expenses, 30% towards discretionary expenses, and 20% towards savings and debt repayment.",
                       "To create a budget, start by setting financial goals for yourself. Then, track your income and expenses for a few months to get a sense of where your money is going.Next, create a budget by allocating your income towards essential expenses, savings and debt repayment, and discretionary expenses.",
                        "Budgeting is all about planning your finances. Consider your goals and adjust your spending accordingly."
                      ]
    },
    {
        "tag": "joke",
        "patterns": [
            "Tell me a joke", "Make me laugh",
            "I need a joke", "Do you know any jokes?"
        ],
        "responses": [
            "Why don't scientists trust atoms? Because they make up everything!",
            "I would tell you a construction joke, but I'm still working on it.",
            "Here's one: I told my computer I needed a break, and it said 'No problem, I'll go to sleep.'"
        ]
    },
    {
        "tag": "time",
        "patterns": [
            "What time is it?", "Tell me the current time", "Current time"
        ],
        "responses": [
            "I'm not able to provide real-time information. Please check your device clock.",
            "I don't have a clock, but your device should have the current time."
        ]
    },
    {
        "tag": "date",
        "patterns": [
            "What's today's date?", "Tell me the date", "Current date"
        ],
        "responses": [
            "I don't have access to real-time information. Please check your device for today's date.",
            "I suggest checking a calendar for the current date."
        ]
    },
    {
        "tag": "news",
        "patterns": [
            "What's the latest news?", "Tell me current news",
            "Any news updates?"
        ],
        "responses": [
            "I'm sorry, I can't provide real-time news updates.",
            "I recommend checking a reputable news website for the latest news."
        ]
    },
    {
        "tag": "restaurant",
        "patterns": [
            "Recommend a restaurant", "Where can I eat?",
            "Looking for a good restaurant", "Food suggestions"
        ],
        "responses": [
            "I can suggest trying local favorites or checking online reviews for highly rated restaurants in your area.",
            "Maybe search for restaurants on Yelp or TripAdvisor to find one that suits your taste."
        ]
    },
    {
        "tag": "travel",
        "patterns": [
            "Plan a trip", "I need travel advice",
            "Where should I travel?", "Travel recommendations"
        ],
        "responses": [
            "Traveling is exciting! Consider what type of destination you prefer: beach, mountains, or cityscape?",
            "I can help with travel suggestions if you provide more details about your interests and budget."
        ]
    },
    {
        "tag": "movies",
        "patterns": [
            "Recommend a movie", "What movie should I watch?",
            "Movie suggestions", "Best films"
        ],
        "responses": [
            "It depends on your taste. Do you like action, comedy, drama, or horror?",
            "Maybe check out recent releases or classic films based on your mood."
        ]
    },
    {
        "tag": "music",
        "patterns": [
            "Recommend a song", "What music should I listen to?",
            "Music suggestions", "Play some music"
        ],
        "responses": [
            "What genre do you prefer? I can suggest something upbeat or mellow.",
            "Maybe try exploring new music on streaming platforms like Spotify or Apple Music."
        ]
    },
    {
        "tag": "education",
        "patterns": [
            "I need study tips", "Help with homework",
            "Educational advice", "Learning resources"
        ],
        "responses": [
            "Setting up a study schedule can help, and don't hesitate to ask for help when needed.",
            "Online resources like Khan Academy, Coursera, or edX can be very useful for learning."
        ]
    },
    {
        "tag": "sports",
        "patterns": [
            "What's the latest in sports?", "Tell me about sports",
            "Sports news", "Favorite sports teams"
        ],
        "responses": [
            "I don't have real-time sports updates, but you can check sports news websites for the latest.",
            "I'm a fan of many sports, though I don't follow a specific team."
        ]
    }
]


In [5]:
# Create the vectorizer and classifier
vectorizer = TfidfVectorizer()
clf = LogisticRegression(random_state=0, max_iter=10000)

#Preprocess the data
tags = []
patterns = []
for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)

In [6]:
# Training the model
x = vectorizer.fit_transform(patterns)
y = tags
clf.fit(x, y)

LogisticRegression(max_iter=10000, random_state=0)

In [102]:
from sklearn.metrics import accuracy_score

y_pred = clf.predict(x)
print("Training Accuracy:", accuracy_score(y, y_pred))

Training Accuracy: 0.95


In [98]:
from sklearn.metrics.pairwise import cosine_similarity

# Python function to chat with the chatbot
def chatbot(input_text):
    input_text_vectorized = vectorizer.transform([input_text])

    # Use both ML classification and similarity matching
    predicted_tag = clf.predict(input_text_vectorized)[0]

    # Cosine Similarity for better matching
    similarities = cosine_similarity(input_text_vectorized, x)
    best_match_index = similarities.argmax()
    best_match_tag = y[best_match_index]

    # Choose the more reliable result
    final_tag = best_match_tag

    for intent in intents:
        if intent['tag'] == final_tag:
            return random.choice(intent['responses'])

    return "Sorry, I didn't understand that."


In [70]:
user_input = "Hello"
response = chatbot(user_input)
print(response)

Hi there, what can I do for you?


In [71]:
user_input = "budget"
response = chatbot(user_input)
print(response)

Budgeting is all about planning your finances. Consider your goals and adjust your spending accordingly.


In [73]:
user_input = "How old are you?"
response = chatbot(user_input)
print(response)

Age is just a number for me.


In [100]:
user_input = "What's your age?"
response = chatbot(user_input)
print(response)

age
I'm as old as the internet!


In [103]:
#making a continuous messaging function
greeting_message=random.choice(intents[0]['responses'][3:])
goodbye_patterns=intents[1]['patterns']
user_input=input(f"{greeting_message}\n")
print(chatbot(user_input))
while(True):
  user_input_message=input()
  if(user_input_message in goodbye_patterns):
    print(chatbot(user_input_message))
    break
  else:
    response=chatbot(user_input_message)
    print(response)


Hey! How can I help you today?
I need help
Sure, I'm here to help. What do you need assistance with?
What's the weather like?
I don't have current weather data, but you can check a weather website or app for up-to-date info.
Tell me a joke
Why don't scientists trust atoms? Because they make up everything!
See ya
Bye! Have a great day!
